In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

# below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

%matplotlib inline
plt.style.use('ggplot')



In [2]:
import psycopg2
user = 'johnchen'
password = 'password'
dbname = 'mimic'
schema = 'mimiciii'
host = 'mimic'
# writing review papers such as: https://arxiv.org/pdf/1911.03977.pdf
query_schema = 'SET search_path to ' + schema + ';'

con = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

# Let's see what we're dealing with 
For now, do this in PSQL prompt. 


In [3]:
# check_all_tables_query = "SELECT * FROM tables;"
# all_tables = pd.read_sql_query(check_all_tables_query,con)

In [19]:
query = query_schema + """
SELECT *
FROM icustays
LIMIT 10
"""
df = pd.read_sql_query(query, con)
df.head()


,row_id,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [22]:
query = query_schema + """
SELECT *
FROM icustays
where subject_id=42210 
"""
df = pd.read_sql_query(query, con)
df.head()


,row_id,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los
0,42074,42210,117994,281393,metavision,SICU,SICU,33,33,2193-02-11 21:32:36,2193-02-19 17:36:09,7.8358


In [23]:
query = query_schema + """
SELECT *
FROM admissions
where subject_id=42210 
"""
df = pd.read_sql_query(query, con)
df.head()


,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,40490,42210,117994,2193-02-11 21:31:00,2193-02-25 17:50:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,SPAN,CATHOLIC,MARRIED,HISPANIC/LATINO - DOMINICAN,2193-02-11 18:39:00,2193-02-11 23:05:00,SUBARACHNIOD HEMORRHAGE,0,1
1,40491,42210,192837,2193-04-10 07:15:00,2193-04-10 22:14:00,None,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicaid,SPAN,CATHOLIC,MARRIED,HISPANIC/LATINO - DOMINICAN,NaT,NaT,CEREBRAL ANEURYSM/SDA,0,0


In [5]:
df.columns

Index(['row_id', 'subject_id', 'hadm_id', 'icustay_id', 'dbsource',
       'first_careunit', 'last_careunit', 'first_wardid', 'last_wardid',
       'intime', 'outtime', 'los'],
      dtype='object')

In [6]:
query = query_schema + """
SELECT *
FROM icustay_detail
LIMIT 10
"""
df = pd.read_sql_query(query, con)
df.head()


,subject_id,hadm_id,icustay_id,gender,dod,admittime,dischtime,los_hospital,age,ethnicity,ethnicity_grouped,admission_type,hospital_expire_flag,hospstay_seq,first_hosp_stay,intime,outtime,los_icu,icustay_seq,first_icu_stay
0,2,163353,243653,M,NaT,2138-07-17 19:04:00,2138-07-21 15:48:00,3.8639,0.0022,ASIAN,asian,NEWBORN,0,1,True,2138-07-17 21:20:07,2138-07-17 23:32:21,0.0918,1,True
1,3,145834,211552,M,2102-06-14,2101-10-20 19:08:00,2101-10-31 13:58:00,10.7847,76.5268,WHITE,white,EMERGENCY,0,1,True,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,1,True
2,4,185777,294638,F,NaT,2191-03-16 00:28:00,2191-03-23 18:41:00,7.7590,47.8450,WHITE,white,EMERGENCY,0,1,True,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,1,True
3,5,178980,214757,M,NaT,2103-02-02 04:31:00,2103-02-04 12:15:00,2.3222,0.0005,ASIAN,asian,NEWBORN,0,1,True,2103-02-02 06:04:24,2103-02-02 08:06:00,0.0844,1,True
4,6,107064,228232,F,NaT,2175-05-30 07:15:00,2175-06-15 16:00:00,16.3646,65.9407,WHITE,white,ELECTIVE,0,1,True,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,1,True


In [7]:
df.columns

# we will definitely need to join it with other variables
# for instance, we need glucose_mean and other variables!

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'dod', 'admittime',
       'dischtime', 'los_hospital', 'age', 'ethnicity', 'ethnicity_grouped',
       'admission_type', 'hospital_expire_flag', 'hospstay_seq',
       'first_hosp_stay', 'intime', 'outtime', 'los_icu', 'icustay_seq',
       'first_icu_stay'],
      dtype='object')

In [8]:
adult_icu_query = query_schema + """
SELECT * from icustays
"""

In [9]:
adult_icu = pd.read_sql_query(query, con)

In [10]:
adult_icu.head()

,subject_id,hadm_id,icustay_id,gender,dod,admittime,dischtime,los_hospital,age,ethnicity,ethnicity_grouped,admission_type,hospital_expire_flag,hospstay_seq,first_hosp_stay,intime,outtime,los_icu,icustay_seq,first_icu_stay
0,2,163353,243653,M,NaT,2138-07-17 19:04:00,2138-07-21 15:48:00,3.8639,0.0022,ASIAN,asian,NEWBORN,0,1,True,2138-07-17 21:20:07,2138-07-17 23:32:21,0.0918,1,True
1,3,145834,211552,M,2102-06-14,2101-10-20 19:08:00,2101-10-31 13:58:00,10.7847,76.5268,WHITE,white,EMERGENCY,0,1,True,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,1,True
2,4,185777,294638,F,NaT,2191-03-16 00:28:00,2191-03-23 18:41:00,7.7590,47.8450,WHITE,white,EMERGENCY,0,1,True,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,1,True
3,5,178980,214757,M,NaT,2103-02-02 04:31:00,2103-02-04 12:15:00,2.3222,0.0005,ASIAN,asian,NEWBORN,0,1,True,2103-02-02 06:04:24,2103-02-02 08:06:00,0.0844,1,True
4,6,107064,228232,F,NaT,2175-05-30 07:15:00,2175-06-15 16:00:00,16.3646,65.9407,WHITE,white,ELECTIVE,0,1,True,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,1,True


In [11]:
# join on hadam (the hospital admissions)
# check: joining in postgresql faster than joining in pandas

hadm_query = query_schema + \\

'''
SELECT * 
from ADMISSIONS
'''


In [12]:
hadm_data = pd.read_sql_query(hadm_query,con)

In [13]:
# we could try and get patient embeddings for the mimic patient as well! Since it seems like we will have to at least get 
# lookups for dealing with the categorical embeddings: religion embedding! 
# actually: in general, we can always get categorical embeddings! (automatically)
# but: categorical embedding for decision trees etc. (do some clustering by country etc.)

# cogsci: religion embedding! 

# even just a cleaning on the diagnosis column would be super useful! 


In [14]:
hadm_data

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,None,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,NaT,NaT,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaT,NaT,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,None,PROTESTANT QUAKER,SINGLE,WHITE,NaT,NaT,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,None,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
5,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,None,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaT,NaT,V-TACH,0,1
6,27,27,134931,2191-11-30 22:16:00,2191-12-03 14:45:00,NaT,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,None,CATHOLIC,None,WHITE,NaT,NaT,NEWBORN,0,1
7,28,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,NaT,NaT,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
8,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,NaT,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,NaT,NaT,UNSTABLE ANGINA\CATH,0,1
9,30,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,None,CATHOLIC,MARRIED,WHITE,NaT,NaT,STATUS EPILEPTICUS,1,1


In [15]:
lab_events_query = query_schema + \\

'''
SELECT * 
FROM LABEVENTS
LIMIT 100
'''

In [16]:
lab_events = pd.read_sql_query(lab_events_query,con) #this query will take a while to run! (probably due to memory size, more than anything)

In [17]:
lab_events.head() # we will need something that maps it back to itemid

,row_id,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,flag
0,281,3,None,50820,2101-10-12 16:07:00,7.39,7.39,units,None
1,282,3,None,50800,2101-10-12 18:17:00,ART,NaN,None,None
2,283,3,None,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,None
3,284,3,None,50804,2101-10-12 18:17:00,22,22.00,mEq/L,None
4,285,3,None,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_roc_curve



ImportError: cannot import name 'plot_roc_curve'

In [ ]:
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer